## Introduction

### Basic RAG method is based on similarity search of vectors and hence most of the times fails to capture the overall syntax of the text presented to it, especially understanding it at higher level. To solve this problem, Knowledge Graph based RAG has been introduced, and Microsoft's GRAPHRAG seems to be a gamechanger, which takes it even further by capturing the entities and their relationship at a very deeperlevel.

### Microsoft GRAPHRAG uses LLM-generated knowledge graphs to provide substantial improvements in question-and-answer performance when conducting document analysis of complex information. A simple LLM generated knowledge graph may not be able to capture the required context of entire document, which has also been showcased via another notebook present in this repo: "llmgraph.ipynb".

### GRAPHRAG powered LLM knowledge graph captures the context as well as indentifies entites, their relationships and places them in appropriate community hierarchy at a much broader level. This notebook discusses the details below.

Importing Required modules

In [1]:
import os
import fitz
import pandas as pd
import tiktoken
# Importing GraphRAG modules

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.vector_stores.lancedb import LanceDBVectorStore

from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch


c:\Code Repo\RAG_Exploration\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Function to convert PDF content into .txt format, as by default GRAPHRAG only supports .txt files currently.

In [2]:
def pdf_to_text(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
 
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
 
 
        doc.close()
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {str(e)}")
        return None

#### Defining folder and file paths.
We have created a directory named 'ragtest' and a subdirectory called 'input' for storing .txt format of pdf file as GRAPHRAG input data. 

### Here, I have taken a PDF preview copy of my book "Concurrency & Parallelism" as input. 

Book is available at amazon: https://www.amazon.in/Concurrency-Parallelism-Efficient-Utilization-Programming/dp/B0D4B1VNVV/ref=sr_1_1?crid=30JAKMUP2AD0K&dib=eyJ2IjoiMSJ9.uWJBGjrM0UXc--Yz5XLgqT9LYQO81oU8GyUmBQnokkilyt2GvitUez3aDbVNZ9cO4NubETRCc140AMlm6sHoKzTP-l7Ur5YAGvENsRHOfAJXGTxPpc5LOo_PaVkTmVV_bEIt7sswOixcptgChCAs_Pn1lm8i6dJ0XLmuaOsdJZh0bUXSeOrUC4hscAcOFSsk_cwvs-noL181HaXSZSkDoE5LgzDgnUDIvQMkh3IQ9FI.ERPgAraYA9cCUqxsZiydyUJCB4hT9QMovlHn_CSTn6c&dib_tag=se&keywords=concurrency+and+parallelism&qid=1720861517&sprefix=%2Caps%2C425&sr=8-1

In [3]:
result_text = pdf_to_text("data/BookPreview.pdf")
result_text
file_name = 'BookPreview.txt'
folder_name= "./ragtest/input/"
# Define the full file path
file_path = os.path.join(folder_name, file_name)

# Write the text to the file in utf-8 encoding format
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(result_text)

print(f'Text has been saved to {file_path}')

Text has been saved to ./ragtest/input/BookPreview.txt


#### GRAPHRAG is one step above than LangChain's LLMGraphTransformer, which converts document text to graph documents.

LLMGraphTransformer only derives Nodes and relationships at a very basic level, however, with GRAPHRAG indexing pipeline, document text is passed through predefined workflows based on DataShaper open-source project. These workflows extract entities, relationships and claims from raw text, and then perform community detection in entities. 

In Datashaper workflows, each step has a verb name and a configuration object. These workflow steps based on 'Verbs' model relational concepts such as SELECT, DROP, JOIN, etc. Details of this relational mapping can be seen here: https://github.com/microsoft/datashaper/blob/main/javascript/schema/src/workflow/verbs.ts

GRAPHRAG's Indexing Pipeline adds custom verbs built on top of the standard relational verbs provided by the DataShaper library, which enable the augmentation of text documents with rich, structured data by utilizing the capabilities of Large Language Models (LLMs). 

For example, in a biomedical research context, the indexing pipeline could be customized to extract entities such as genes, proteins, and diseases, along with relationships like gene-disease associations or protein-protein interactions. Claims could be extracted in the form of research findings or hypotheses, and community structures could be used to identify clusters of related research topics or collaborative networks of researchers.

More details of dataflow can read at: https://microsoft.github.io/graphrag/posts/index/1-default_dataflow/


### Hierarchical Leiden Algorithm : For Community Detection

One of the important steps in GRAPHRAG, after entity and relationship extraction, is to generate a hierarchy of entity communities using the Hierarchical Leiden Algorithm. Community detection involves identifying groups of nodes in a network that are more densely connected internally than with the rest of the network. It is helpful for understanding the structure and function of networks.

The Hierarchical Leiden algorithm builds on the already popular Louvain method for community detection but introduces several improvements to enhance accuracy, efficiency, and stability.

The Leiden algorithm consists of three phases: (1) local moving of nodes, (2) refinement of the partition and (3) aggregation of the network based on the refined partition, using the non-refined partition to create an initial partition for the aggregate network. It recursively merges communities into single nodes by greedily optimizing the modularity and the process repeats in the condensed graph. 
The algorithm moves individual nodes from one community to another to find a partition, which is then refined. An aggregate network is created based on the refined partition, using the non-refined partition to create an initial partition for the aggregate network.

###### More about this algorithm can be read at: https://arxiv.org/pdf/1810.084733


#### How GRAPHRAG buils its knowledge graph has been shown in this flowchart: 
https://tinyurl.com/2s4ytcur

Courtsey: "Prompt Engineering Youtube Channel"

### Graph RAG comes with a retrieval module which extracts relevant information from document chunks, called as Query Engine. It performs two types of search: Local Search and Global Search.

Local Search is more relevant if we want to extract information about specific entities mentioned in the document. It combines relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. Local Search responds to user queries by identifying semantically-related entities and extracting relevant details from the knowledge graph and associated raw documents, all within a defined context window.

The identified entities serve as initial access points into the knowledge graph. From these entities, Local Search can traverse the graph to extract further relevant details such as connected entities, relationships, entity covariates (attributes), and community reports. The extracted entities, relationships, and text chunks are prioritized based on their relevance to the user query. It is ensured that the selected data fits within a single context window of pre-defined size, optimizing it for response generation.

Global Search is about extracting and understanding the document on a higher level like extracting a summarized information from the document or getting aggregated information such as "How many chapters are there in the book?". Basic RAG performs terribly here because it relies on a vector search of semantically similar text content within the dataset. There is nothing in the query to direct it to the correct information.

Using Global search, the LLMs use meaningful semantic clusters of our data, that are pre-summarized, to extract higher level information for a user query. It uses a collection of LLM-generated community reports from a specified level of the graph's community hierarchy as context data to generate response in a map-reduce manner. 


### Run GRAPHRAG Indexing pipeline.

In the terminal, run the following command to set up required folders and configuration files for GRAPHRAG.

> python -m graphrag.index --init --root ./ragtest

This will create two files: .env and settings.yaml in the ./ragtest directory. Mention your OpenAI/AzureOpenAI API-KEY in .env file and respective azure hosting details in settings.yaml

Once done with the step, run the following command in the terminal to start GRAPHRAG Indexing pipeline.

> python -m graphrag.index --root ./ragtest


The indexing pipeline will perform the workflows for entities and relationship extraction as well as community detetction and store outputs in parquet file formats in the artifacts directory of output folder. We extract the artifacts in a seperate folder 'final_artifacts' under 'ragtest' for ease of processing further for question answering on the document.

It contains files such as "create_final_entities.paraquet", "create_final_relationships.paraquet", etc.

In [4]:
INPUT_DIR = "./ragtest/final_artifacts"
LANCEDB_URI = "./lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [5]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 777


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,top_level_node_id,x,y
0,0,"""REESHABH CHOUDHARY""","""PERSON""",Reeshabh Choudhary is a prominent author and c...,"05f0679d2dc1829d5e6a505b4d5bdedd,0b4d01189034f...",5,25,0,b45241d70f0e43fca764df95b2b81f77,25,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,"""CONCURRENCY & PARALLELISM""","""EVENT""","""Concurrency & Parallelism"" refers to concepts...","0b4d01189034f316cbd1f2f36add64df,21f6c5fd69061...",2,6,1,4119fd06010c494caa07f439b333f4c5,6,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,"""MAA SARASWATI""","""PERSON""",Maa Saraswati is a deity revered for her assoc...,"05f0679d2dc1829d5e6a505b4d5bdedd,3577d131f8ee4...",5,1,2,d3835bf3dda84ead99deadbeac5d0d7d,1,None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,"""GANESH JI""","""PERSON""",Ganesh ji is a deity known for removing obstac...,"05f0679d2dc1829d5e6a505b4d5bdedd,3577d131f8ee4...",5,1,3,077d2820ae1845bcbb1803379a3d1eae,1,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,"""HANUMAN JI""","""PERSON""","Hanuman Ji is a deity known for his strength, ...","05f0679d2dc1829d5e6a505b4d5bdedd,3577d131f8ee4...",5,1,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,1,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0


In [6]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 266


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,"""REESHABH CHOUDHARY""","""CONCURRENCY & PARALLELISM""",2.0,Reeshabh Choudhary is the author of the docume...,"[0b4d01189034f316cbd1f2f36add64df, 927dbe44806...",dbe9063124d047dc8d6fcaeadcda038f,0,25,6,31
1,"""REESHABH CHOUDHARY""","""MAA SARASWATI""",2.0,"""Reeshabh Choudhary acknowledges Maa Saraswati...","[05f0679d2dc1829d5e6a505b4d5bdedd, 3577d131f8e...",c885166d0c454a748376b56279f96408,1,25,1,26
2,"""REESHABH CHOUDHARY""","""GANESH JI""",2.0,"""Reeshabh Choudhary acknowledges Ganesh ji for...","[05f0679d2dc1829d5e6a505b4d5bdedd, 3577d131f8e...",586bccefb1e344289c1ee984e165de9c,2,25,1,26
3,"""REESHABH CHOUDHARY""","""HANUMAN JI""",2.0,Reeshabh Choudhary acknowledges Hanuman Ji for...,"[05f0679d2dc1829d5e6a505b4d5bdedd, 3577d131f8e...",a2201b8753ba4847ab0b22054e27d2c0,3,25,1,26
4,"""REESHABH CHOUDHARY""","""OBJECTS, DATA & AI""",1.0,"""Reeshabh Choudhary reflects on his journey fr...",[3577d131f8ee44088b150e425fbbc8be],b5ecd0553dd742f5813c9b855d548a41,4,25,1,26


In [7]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 46


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,43,# CPU and Its Core Components\n\nThe community...,2,8.5,CPU and Its Core Components,The impact severity rating is high due to the ...,The community revolves around the CPU (Central...,[{'explanation': 'The CPU is the central entit...,"{\n ""title"": ""CPU and Its Core Components"",...",7fcc3131-4e26-4d7d-9cc3-694042a464a4
1,44,# CPU Interrupts and Interrupt Vector Table\n\...,2,8.5,CPU Interrupts and Interrupt Vector Table,The impact severity rating is high due to the ...,"The community revolves around the CPU, Interru...",[{'explanation': 'Interrupts are crucial signa...,"{\n ""title"": ""CPU Interrupts and Interrupt ...",7f677833-1786-4e30-8a21-0f7772d2102d
2,45,# CPU Interrupt Handling and Context Switching...,2,8.5,CPU Interrupt Handling and Context Switching,The impact severity rating is high due to the ...,The community revolves around the CPU's handli...,[{'explanation': 'The Interrupt Service Routin...,"{\n ""title"": ""CPU Interrupt Handling and Co...",a02c0752-4e09-42bc-a9b2-cc054b3b79f6
3,12,# Evolution of Modern CPU Architectures\n\nThe...,1,8.5,Evolution of Modern CPU Architectures,The impact severity rating is high due to the ...,The community revolves around the evolution of...,[{'explanation': 'Transistors are semiconducto...,"{\n ""title"": ""Evolution of Modern CPU Archi...",99c09e70-520e-4cd0-a262-190ba8aca851
4,13,# CPU and Logic Gates\n\nThe community revolve...,1,8.5,CPU and Logic Gates,The impact severity rating is high due to the ...,The community revolves around the CPU and its ...,[{'explanation': 'Logic gates are the fundamen...,"{\n ""title"": ""CPU and Logic Gates"",\n ""s...",e0166068-6ae8-4ed5-af69-6ae7e5d35cc1


In [8]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 72


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,0b4d01189034f316cbd1f2f36add64df,\n \n \nReeshabh Choudhary \n \n \nCONCURRENC...,300,[493ecc11169261830bf2b46755085fa6],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...",[dbe9063124d047dc8d6fcaeadcda038f]
1,3577d131f8ee44088b150e425fbbc8be,्रभृततभभदेव ैः सदा वण्न्दता \nसा मां पातु सरस्...,300,[493ecc11169261830bf2b46755085fa6],"[b45241d70f0e43fca764df95b2b81f77, d3835bf3dda...","[c885166d0c454a748376b56279f96408, 586bccefb1e..."
2,05f0679d2dc1829d5e6a505b4d5bdedd,me and remove my ignorance \ncompletely. \n \...,300,[493ecc11169261830bf2b46755085fa6],"[b45241d70f0e43fca764df95b2b81f77, d3835bf3dda...","[c885166d0c454a748376b56279f96408, 586bccefb1e..."
3,a433e2c40be3229a0521b18d8a0db2fd,NEC is usually detected in pre-\nmatured babi...,300,[493ecc11169261830bf2b46755085fa6],"[27f9fbe6ad8c4a8b9acee0d3596ed57c, e1fd0e904a5...","[b232fb0f2ac14790b931d1e7fcddd8ad, 1c16b22e18d..."
4,1edfe8ac6923033790c284f36216af4b,"the mind, and make the heart beat violently, ...",300,[493ecc11169261830bf2b46755085fa6],"[9646481f66ce4fd2b08c2eddda42fc82, d91a266f766...","[71a0a8c1beb64da08124205e9a803d98, f84314943be..."


In [9]:
deployment_name="LLM AZURE DEPLOYMENT NAME"
openai_api_type="azure"
api_key="API-KEY"
api_version="2024-02-15-preview"
azure_endpoint="BASE URL"
embedding_model = "EMBEDDING MODEL NAME"
embedding_deployment = "DEPLOYMENT NAME"

In [10]:
llm = ChatOpenAI(
    api_key=api_key,
    model="LLM Model Name",
    deployment_name=deployment_name,
    api_base=azure_endpoint,
    api_version=api_version,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

In [11]:
token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=azure_endpoint,
    api_type=OpenaiApiType.AzureOpenAI,
    api_version="2024-02-01",
    model=embedding_model,
    deployment_name=embedding_deployment,
    max_retries=20,
)

Configuring Local Search Context

In [12]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [13]:
local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

Setting up Local Search Query Engine

In [24]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

Let us now explore the result of a question specific to internal details of the book by Local Search and the result of same question by Global Search, and the difference will be quite clear.

In [34]:
result = await search_engine.asearch("List the chapters mentioned in the Contents section.")
print(result.response)

The contents section of the book mentions several chapters and sections that cover a wide range of topics related to concurrency, parallelism, operating systems, and application development. Here are the chapters and sections mentioned:

### Part 1: OS & Application Development

1. **Introduction**
   - Concurrency Vs Parallelism

2. **Beneath the Surface: OS**
   - Multiple Processor System
   - Multi-Core System
   - Use of Thread Pools
   - Multi-threading and Parallel Programming
   - Adoption of GPUs
   - GPU Architecture
   - Summary

3. **Order to Chaos: Synchronization**
   - Race Condition
   - Synchronization Strategies
     - Hardware Level Solutions
     - Atomic Operations
     - Software Level Solutions
   - Cost and Considerations about Synchronization
   - Practical Scenario
   - Summary

4. **Beyond Thread Synchronization**
   - Asynchronous Approach
     - Use of Event Loop
     - Asynchronous Nature of Node JS
     - Curious Case of Redis
     - Limitations of Asynch

Let us now configure Global Search context

In [17]:
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

In [26]:
global_context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

In [27]:
global_context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

Setting up Global Search Query Engine

In [28]:
global_search_engine = GlobalSearch(
    llm=llm,
    context_builder=global_context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=global_context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [33]:
result = await global_search_engine.asearch("List the chapters mentioned in the Contents section.")
print(result.response)

Here are the chapters mentioned in the Contents section:

### CPU and Its Core Components
This chapter delves into the fundamental components of the CPU, including its architecture and key functionalities [Data: Reports (43)].

### Concurrency and Parallelism in Computing Systems
This chapter covers the principles of concurrency and parallelism, essential for modern computing systems [Data: Reports (33, 23)].

### Process Control Block (PCB) and Related Components
This chapter discusses the Process Control Block (PCB) and its role in managing processes within an operating system [Data: Reports (38)].

### Intel and Moore's Law
This chapter explores Intel's contributions to the computing industry and the implications of Moore's Law [Data: Reports (40)].

### Evolution of Modern CPU Architectures
This chapter traces the development and advancements in CPU architectures over time [Data: Reports (12, 26)].

### Maurice Herlihy and Nir Shavit in Concurrent Programming
This chapter highlight

It can be evidently seen that Global search fails to explictly specify the chapter names mentioned in the Contents section of the book,however, it has an overall context around what the topic of discussion is about.